In [1]:
from class_crawler import Crawler

In [81]:
# importing libraries

from bs4 import BeautifulSoup
import urllib.request
import re

In [82]:
import pandas as pd

In [8]:
dir_ = "wiki_links/"

In [5]:
list_files = ['wirusowe zapalenia watroby.txt', 'wirusowe_choroby_roslin.txt', 
              'choroby_wirusowe.txt', "goraczki_krwotoczne.txt", "grypa.txt", 
              "choroby_pasorzytnicze.txt"]

In [9]:
crawler1 = Crawler()

In [11]:
links_from_files = crawler1.read_files(list_files, dir_)

In [14]:
links_from_files[0][:5]

['https://pl.wikipedia.org//wiki/Antygen_HBs\n',
 'https://pl.wikipedia.org//wiki/Wirusowe_zapalenie_w%C4%85troby_typu_B\n',
 'https://pl.wikipedia.org//wiki/Wirusowe_zapalenie_w%C4%85troby_typu_C\n',
 'https://pl.wikipedia.org//wiki/Wirusowe_zapalenie_w%C4%85troby_typu_E\n']

In [74]:
acronyms_data_frame = pd.DataFrame(columns=["akronim", "fraza polska", "fraza obca", "język", "hasło_wikipedia"])

In [75]:
acronyms_data_frame

,akronim,fraza polska,fraza obca,język,hasło_wikipedia


In [76]:
# list_all_links

In [77]:
def readAll(lines_links, acronyms_data_frame):
    for linklist in lines_links:
        
        for link in linklist:
            link = link[:-1]
            series_acronym = getAcronymFromPage(link)
            print(series_acronym)
            acronyms_data_frame = acronyms_data_frame.append(series_acronym, ignore_index=True)
        
    return acronyms_data_frame

In [78]:
acronyms_data_frame = readAll(list_all_links, acronyms_data_frame)

akronim                                           HB
fraza polska                             Antygen HBs
fraza obca         wirusowe zapalenie wątroby typu B
język                                    wywołującym
hasło_wikipedia                          Antygen_HBs
dtype: object
akronim                                               WZW
fraza polska           Wirusowe zapalenie wątroby typu B 
fraza obca                                         typu B
język                                                 WZW
hasło_wikipedia    Wirusowe_zapalenie_w%C4%85troby_typu_B
dtype: object
akronim                                               WZW
fraza polska           Wirusowe zapalenie wątroby typu C 
fraza obca                                              C
język                                                 WZW
hasło_wikipedia    Wirusowe_zapalenie_w%C4%85troby_typu_C
dtype: object
akronim                                               WZW
fraza polska            Wirusowe zapalenie wątroby typu

In [92]:
acronyms_data_frame[0:10]

,akronim,fraza polska,fraza obca,język,hasło_wikipedia
0,HB,Antygen HBs,wirusowe zapalenie wątroby typu B,wywołującym,Antygen_HBs
1,WZW,Wirusowe zapalenie wątroby typu B,typu B,WZW,Wirusowe_zapalenie_w%C4%85troby_typu_B
2,WZW,Wirusowe zapalenie wątroby typu C,C,WZW,Wirusowe_zapalenie_w%C4%85troby_typu_C
3,WZW,Wirusowe zapalenie wątroby typu E,hepatitis E,ang,Wirusowe_zapalenie_w%C4%85troby_typu_E
4,brak,Wirusowe choroby roślin,brak,brak,Wirusowe_choroby_ro%C5%9Blin
5,brak,Chloroza nerwów liści maliny,vein chlorosis virus,Raspberry,Chloroza_nerw%C3%B3w_li%C5%9Bci_maliny
6,RBDV,Krzaczasta karłowatość maliny,brak,RBDV,Krzaczasta_kar%C5%82owato%C5%9B%C4%87_maliny
7,brak,Mozaika maliny,Raspberry mosaic virus,ang,Mozaika_maliny
8,RNA,Mozaika tytoniowa,Nicotiana virus,łac,Mozaika_tytoniowa
9,BBS,Oparzelina borówki wysokiej,corymbosum,Vaccinium,Oparzelina_bor%C3%B3wki_wysokiej


In [80]:
acronyms_data_frame.to_csv (r'medical_from_wiki.csv', index = None, header=True)

In [26]:
acronyms_data_frame

,akronim,fraza polska,fraza obca,język,hasło_wikipedia


In [16]:
def getAcronymFromPage(url):
    
    page = urllib.request.urlopen(url)

    soup = BeautifulSoup(page, 'html.parser')

    wikiEnd = re.search(r'\/wiki\/.+', url)
    wikiEnd = wikiEnd.group()[6:]

    header_section = soup.find_all('p')
    
    if len(header_section[0]) < 6:
        header_section = header_section[1]
        
    else:
        header_section = header_section[0]

    text_work = header_section.getText()

#     print(text_work)
    matchObjExtension = re.match(r'(\w|\s)+', text_work, re.UNICODE)


    matchObjAcronym = re.search(r'[A-Z][A-Z]+', text_work) # ogranicz do pierwszego akapitu.
    
    if matchObjAcronym == None:
        matchObjAcronym = "brak"
    
    else:
        matchObjAcronym = matchObjAcronym.group()

    matchObjTranslation = re.search(r'\(\w\w\w\..(\w|\s)+', text_work)
    
    if matchObjTranslation == None:
        matchObjTranslation = re.search(r'\((\w|\s)+', text_work)
#     matchObjTranslation = re.search(r'\w\w\w.\xa0(\w|\s)+', text_work)

    if matchObjTranslation == None:
        
        print("no translation")
        matchObjLanguage = "brak"
        matchObjTranslationProper = "brak"
        
        
    else:
#         print("dupa")
#         print(matchObjTranslation)

        matchObjLanguage = re.search(r'\w+', matchObjTranslation.group())
#         print(matchObjLanguage)
        matchObjLanguage = matchObjLanguage.group()
        matchObjTranslationProper = re.search(r'\s(\w|\s)+', matchObjTranslation.group())

#         print(matchObjTranslationProper)
        if matchObjTranslationProper == None:
            matchObjTranslationProper = "brak"
        else:
            matchObjTranslationProper = matchObjTranslationProper.group()[1:]
    
    series_acronym = pd.Series([matchObjAcronym, matchObjExtension.group(), 
                                matchObjTranslationProper, matchObjLanguage, wikiEnd], 
                               index=["akronim", "fraza polska", "fraza obca", "język", "hasło_wikipedia"])
    return series_acronym
    

In [14]:
series_acronym = getAcronymFromPage(url)

Wirusowe zapalenie wątroby, WZW (łac. hepatitis viralis[1]) – choroba wątroby wywołana zakażeniem wirusowym. Często potocznie nazywana „żółtaczką”, jest to jednak określenie nieprawidłowe i niemedyczne (w medycynie termin ten oznacza jedynie objaw zażółcenia powłok skórnych) oraz nieścisłe, z uwagi na różnorodny przebieg wirusowych zapaleń wątroby (które mogą przebiegać także bez zażółcenia skóry).

dupa
<re.Match object; span=(32, 55), match='(łac.\xa0hepatitis viralis'>
<re.Match object; span=(1, 4), match='łac'>
<re.Match object; span=(5, 23), match='\xa0hepatitis viralis'>


In [15]:
acronyms_data_frame.append(series_acronym, ignore_index=True)

,akronim,fraza polska,fraza obca,język,hasło_wikipedia
0,WZW,Wirusowe zapalenie wątroby,hepatitis viralis,łac,Wirusowe_zapalenie_w%C4%85troby
